**Budget Tracker GUI**

In [ ]:
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox
from tkcalendar import DateEntry

# Initialize main window
root = tk.Tk()
root.title("Budget Tracker")

# Category list to store default and user-added categories
categories = ["Food", "Transport", "Rent", "Other"]

# List to store expenses
expenses = []
budget = 0
remaining_budget = tk.StringVar(value="Set your monthly budget")

# Function to set the monthly budget
def set_budget():
    global budget, remaining_budget
    try:
        budget = float(simpledialog.askstring("Set Budget", "Enter your monthly budget:"))
        remaining_budget.set(f"Remaining Budget: {budget}")
        messagebox.showinfo("Budget Set", f"Your budget has been set to {budget}")
    except ValueError:
        messagebox.showwarning("Invalid Input", "Please enter a valid number for the budget.")

# Function to update remaining budget after adding expense
def update_remaining_budget(amount):
    global budget
    try:
        amount = float(amount)
        remaining = budget - sum(float(exp[1]) for exp in expenses)
        remaining_budget.set(f"Remaining Budget: {remaining}")
        give_savings_advice(remaining)
    except ValueError:
        messagebox.showwarning("Invalid Input", "Invalid amount entered.")

# Function to give savings advice based on remaining budget
def give_savings_advice(remaining):
    if remaining <= 0:
        advice_label.config(text="You've exceeded your budget. Try to save more next month.")
    elif remaining <= 0.2 * budget:
        advice_label.config(text="Caution: You're close to exceeding your budget!")
    elif remaining >= 0.5 * budget:
        advice_label.config(text="Good job! You’re on track to save well this month.")
    else:
        advice_label.config(text="You're doing fine. Keep an eye on your spending!")

# Date picker
tk.Label(root, text="Date").grid(row=1, column=0)
date_entry = DateEntry(root, width=12, background='darkblue', foreground='white', borderwidth=2)
date_entry.grid(row=1, column=1)

# Amount field
tk.Label(root, text="Amount").grid(row=2, column=0)
amount_entry = tk.Entry(root)
amount_entry.grid(row=2, column=1)

# Category dropdown
tk.Label(root, text="Category").grid(row=3, column=0)
category_entry = ttk.Combobox(root, values=categories)
category_entry.grid(row=3, column=1)

# Description field
tk.Label(root, text="Description").grid(row=4, column=0)
description_entry = tk.Entry(root)
description_entry.grid(row=4, column=1)

# Function to add a new category
def add_category():
    new_category = simpledialog.askstring("Add Category", "Enter new category:")
    if new_category:
        # Check if the category already exists
        if new_category not in categories:
            categories.append(new_category)
            category_entry['values'] = categories  # Update the dropdown values
            messagebox.showinfo("Success", f"Category '{new_category}' added successfully!")
        else:
            messagebox.showwarning("Warning", f"Category '{new_category}' already exists.")

# Button to add a new category
add_category_button = tk.Button(root, text="Add Category", command=add_category)
add_category_button.grid(row=3, column=2)  # Place next to the category dropdown

# Function to add expense
def add_expense():
    date = date_entry.get()
    amount = amount_entry.get()
    category = category_entry.get()
    description = description_entry.get()
    
    # Validate input and add to the expense list
    if date and amount and category and description:
        try:
            expenses.append((date, float(amount), category, description))
            update_remaining_budget(amount)
            messagebox.showinfo("Success", "Expense added successfully!")
        except ValueError:
            messagebox.showwarning("Input Error", "Please enter a valid number for amount.")
    else:
        messagebox.showwarning("Input Error", "Please fill all fields")

# Button to save the expense
add_button = tk.Button(root, text="Add Expense", command=add_expense)
add_button.grid(row=5, column=1)

# Function to view expenses
def view_expenses():
    # Create a new window to show expenses
    view_window = tk.Toplevel(root)
    view_window.title("View Expenses")
    
    # Treeview widget to display expenses
    columns = ("Date", "Amount", "Category", "Description")
    tree = ttk.Treeview(view_window, columns=columns, show="headings")
    tree.heading("Date", text="Date")
    tree.heading("Amount", text="Amount")
    tree.heading("Category", text="Category")
    tree.heading("Description", text="Description")
    
    # Insert all expenses into the tree
    for expense in expenses:
        tree.insert("", "end", values=expense)
    
    tree.pack(expand=True, fill="both")

# Button to view expenses
view_button = tk.Button(root, text="View Expenses", command=view_expenses)
view_button.grid(row=6, column=1)

# Section to display budget and savings advice
tk.Label(root, text="Budget and Advice").grid(row=0, column=0, columnspan=2)
budget_button = tk.Button(root, text="Set Budget", command=set_budget)
budget_button.grid(row=0, column=2)

remaining_label = tk.Label(root, textvariable=remaining_budget)
remaining_label.grid(row=7, column=0, columnspan=2)

advice_label = tk.Label(root, text="Savings advice will appear here.")
advice_label.grid(row=8, column=0, columnspan=2)

root.mainloop()
